In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from visualization_utilities import *
from registration_utilities import *
from annotation_utilities import *

Setting environment for AWS compute node


No vtk


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 0
Child returned 1
0.35 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.35 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.33 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


# Compare different classifiers

In [2]:
stack = 'MD635'

In [3]:
setting1 = 2
setting2 = 10
# descriptive1 = 'Nissl-trained'
# descriptive2 = 'NTB-trained'

In [4]:
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [5]:
df, _ = DataManager.load_annotation_v3(stack=stack)
sections_containing_structure = {structure: set(df[df['name']==structure]['section'].values) for structure in all_known_structures}

'No object named structures in the file'


Annotation has no structures.


In [12]:
for structure in all_known_structures:
    
    print structure
    
    if structure in sections_containing_structure:
        sections = sections_containing_structure[structure]
    else:
#         sections = range(first_sec, last_sec + 1)
        continue
    
    for sec in sections:
        fn = metadata_cache['sections_to_filenames'][stack][sec]
        
        try:
            viz1 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting1))
            viz2 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting2))

            cv2.putText(viz1, '%d %s %d' % (sec, structure, setting1), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            cv2.putText(viz2, '%d %s %d' % (sec, structure, setting2), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)

            viz = np.hstack([viz1, viz2])

            viz_fp = os.path.join('/home/yuncong/csd395/CSHL_scoremap_viz_juxtaposed', structure, stack, fn + '_setting_%d_vs_setting_%d.jpg' % (setting1, setting2))
            create_if_not_exists(os.path.dirname(viz_fp))

            imsave(viz_fp, viz)
        except:
            continue

5N
6N
7N
7n
Amb
LC
LRt
Pn
Tz
VLL
RMC
SNC
SNR
3N
4N
Sp5I
Sp5O
Sp5C
PBG
10N
VCA
VCP
DC
AP
12N
RtTg
sp5
outerContour
SC
IC


In [ ]:
for sec in range(first_sec, last_sec + 1):
    
    fn = metadata_cache['sections_to_filenames'][stack][sec]
    if is_invalid(fn):
        continue
        
    for structure in all_known_structures:
        
        try:
            viz1 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting1))
            viz2 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting2))

            cv2.putText(viz1, '%d %s %d' % (sec, structure, setting1), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            cv2.putText(viz2, '%d %s %d' % (sec, structure, setting2), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            
            viz = np.hstack([viz1, viz2])

            viz_fp = os.path.join('/home/yuncong/csd395/CSHL_scoremap_viz_juxtaposed', structure, stack, fn + '_setting_%d_vs_setting_%d.jpg' % (setting1, setting2))
            create_if_not_exists(os.path.dirname(viz_fp))

            imsave(viz_fp, viz)
        except:
            continue

# Compare Different Warpings

In [2]:
stack_moving = 'atlasV2'
stack_fixed = 'MD642'

In [3]:
classifier_setting = 2

In [9]:
warp_setting1= 1
warp_setting2 = 5
warp_setting3 = 12

In [10]:
basename1 = DataManager.get_warped_volume_basename(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting1)

basename2 = DataManager.get_warped_volume_basename(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting2)

basename3 = DataManager.get_warped_volume_basename(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting3)


sidebyside_basename = '_sideByside_'.join([basename1, basename2, basename3])

In [11]:
first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]

In [16]:
for sec in range(first_sec, last_sec+1):
# for sec in [155]:
    
    if is_invalid(stack=stack_fixed, sec=sec):
        continue
    
    viz1_fn = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting1,
                                      section=sec)
    viz1 = imread(viz1_fn)
    
    viz2_fn = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting2,
                                      section=sec)
    viz2 = imread(viz2_fn)
    
    viz3_fn = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting3,
                                      section=sec)
    viz3 = imread(viz3_fn)
    
    ############################
    viz = np.hstack([viz1, viz2, viz3])
    
    viz_fp = os.path.join('/home/yuncong/csd395/CSHL_registration_viz_juxtaposed', 
                          sidebyside_basename,
                          sidebyside_basename + '_%04d.jpg' % sec)
    
    create_parent_dir_if_not_exists(viz_fp)
    imsave(viz_fp, viz)